
# OpenAI ChatModel Integration — Test Notebook

This notebook verifies your `ChatModel` abstraction using the **OpenAI** implementation (`OpenAIChat`).  
It includes smoke tests for: basic chat, streaming, JSON mode, and tool-calls.  
> **Prereqs**: You have your package installed (e.g., `pip install -e .`) and an OpenAI API key.



## 1) Environment & Package Setup

- Ensure you are at your project root (where `pyproject.toml` lives).
- (Once) install your package locally:  
  ```bash
  pip install -e .
  ```
- Make sure you have the OpenAI SDK installed (your `pyproject.toml` should include `openai>=1.45.0`).  
  Otherwise:
  ```bash
  pip install openai>=1.45.0
  ```
- Provide your API Key (either export the env var before launching Jupyter, or set it in the cell below).


In [ ]:

import os

# Option A: already set in your shell
# os.environ['OPENAI_API_KEY']

# Option B: set here for this notebook session
os.environ['OPENAI_API_KEY'] = "put your open ai api key here"

print("OPENAI_API_KEY set:", bool(os.getenv("OPENAI_API_KEY")))


OPENAI_API_KEY set: True



## 2) Imports


In [2]:

from src.db_crawl_agents.llms.openai_integration.chat_openai import OpenAIChat
from src.db_crawl_agents.utils.types import ChatMessage



## 3) Basic Chat (non-streaming)

This will call your `OpenAIChat.chat(...)` method with a tiny prompt and print the response.


In [3]:

chat = OpenAIChat(model="gpt-4o")

messages = [
    ChatMessage(role="system", content="You are a concise assistant."),
    ChatMessage(role="user", content="Give me two short facts about Bengaluru.")
]

resp = chat.chat(messages, temperature=0.2, max_tokens=150)
print("MODEL  :", resp.model)
print("FINISH :", resp.finish_reason)
print("USAGE  :", resp.usage)
print("\nASSISTANT:\n", resp.content)


MODEL  : gpt-4o-2024-08-06
FINISH : stop
USAGE  : {'completion_tokens': 68, 'prompt_tokens': 25, 'total_tokens': 93, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}

ASSISTANT:
 1. Bengaluru, often referred to as the "Silicon Valley of India," is a major hub for information technology and innovation, hosting numerous tech companies and startups.

2. The city is known for its pleasant climate, with moderate temperatures throughout the year, earning it the nickname "Garden City" due to its many parks and green spaces.



## 4) Streaming Chat

This yields token deltas in real time and then prints the final aggregated text once the stream ends.


raise issue: streaming is not functional right now I will need to debug this

In [4]:

chat = OpenAIChat(model="gpt-4o")

messages = [
    ChatMessage(role="system", content="You are a streaming assistant."),
    ChatMessage(role="user", content="Stream one short paragraph about Hampi hospitality and architecture.")
]

print("STREAM (tokens):", end=" ")
final_text = []
for delta in chat.stream(messages, temperature=0.2, max_tokens=120):
    # Intermediate deltas have small 'content' chunks; final one has the full aggregate
    print(delta.content, end="", flush=True)
    final_text.append(delta.content)
print("\n\nFINAL:")
print("".join(final_text))


STREAM (tokens): 

TypeError: 'Responses' object is not callable


## 5) JSON Mode (structured output)

Request a strict JSON object using `response_format={"type": "json_object"}`.  
The model should return a valid JSON string in `resp.content`.


In [5]:

import json

chat = OpenAIChat(model="gpt-4o")

messages = [
    ChatMessage(role="system", content="You are a JSON-only assistant."),
    ChatMessage(role="user", content="Return a JSON with keys: city, highlights (list of 2 strings) for 'Mysuru'.")
]

resp = chat.chat(
    messages,
    temperature=0.0,
    response_format={"type": "json_object"},
    max_tokens=200,
)

print("RAW:", resp.content)
try:
    obj = json.loads(resp.content)
    print("\nParsed JSON:", obj)
except json.JSONDecodeError as e:
    print("❌ JSON parse error:", e)


RAW: {
  "city": "Mysuru",
  "highlights": [
    "Mysore Palace",
    "Brindavan Gardens"
  ]
}

Parsed JSON: {'city': 'Mysuru', 'highlights': ['Mysore Palace', 'Brindavan Gardens']}



## 6) Tool Calling (function-calling)

We define a simple tool schema and ask the model to call it.  
> **Note**: This only checks that the model **returns** a tool call; it does **not** execute the tool.  
You can wire an execution loop in your app to run the tool and then provide a `tool` message back to the model.


In [6]:

# Define a simple tool/function for the model to call
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the current weather for a city (mock).",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {"type": "string", "description": "City name"}
                },
                "required": ["city"]
            }
        }
    }
]

chat = OpenAIChat(model="gpt-4o")

messages = [
    ChatMessage(role="system", content="You are a helpful assistant. Use tools if needed."),
    ChatMessage(role="user", content="What's the weather in Bengaluru right now?")
]

resp = chat.chat(
    messages,
    tools=tools,
    tool_choice="auto",
    temperature=0.2,
    max_tokens=200
)

print("ASSISTANT:", resp.content)
print("TOOL CALLS:", resp.tool_calls)


ASSISTANT: 
TOOL CALLS: [ToolCall(id='call_BILCjxsxnzJXgWU1YnBqtPTp', type='function', function_name='get_weather', arguments_json='{"city":"Bengaluru"}')]



### (Optional) Round-trip: Provide a tool result back to the model

If the previous cell returned a function call, you would:
1. Parse `resp.tool_calls[0].arguments_json`  
2. Execute your function (server-side)  
3. Add a **tool** message with the result and call the model again

Below is a mock example to illustrate that pattern.


In [7]:

import json
from typing import Optional

def execute_get_weather(city: str) -> str:
    # Mock result (replace with real integration)
    return f"It is 29°C and partly cloudy in {city}."

def to_openai_tool_calls(tool_calls):
    return [{
        "id": tc.id,
        "type": "function",
        "function": {"name": tc.function_name, "arguments": tc.arguments_json or "{}"},
    } for tc in (tool_calls or [])]


assistant_with_calls = ChatMessage(
    role="assistant",
    content=resp.content or "",
    tool_calls=to_openai_tool_calls(resp.tool_calls)  # ✅ put calls here
)
followup_messages = messages + [assistant_with_calls]
for tc in resp.tool_calls or []:
    args = json.loads(tc.arguments_json or "{}")
    result = execute_get_weather(args.get("city", "unknown")) if tc.function_name == "get_weather" else "Unknown tool"
    followup_messages.append(
        ChatMessage(role="tool", content=result, name=tc.function_name, tool_call_id=tc.id)
    )

final_resp = chat.chat(followup_messages)
print(final_resp.content)

The current weather in Bengaluru is 29°C and partly cloudy.



---

## ✅ Done

You just validated:
- Basic chat (`chat`)
- Streaming (`stream`)
- JSON mode (`response_format={"type":"json_object"}`)
- Tool calling round-trip pattern

If you want, duplicate this notebook and swap in other providers (Anthropic, Groq, Bedrock) using the same `ChatModel` interface.


In [8]:
import openai
print(openai.__version__)

1.107.2
